In [5]:
import cv2 as cv
import pandas as pd
import os
import matplotlib.pyplot as plt

# =========================================
# 1️⃣ Chemins
# =========================================
base_dir = os.getcwd()

matricule_folder = os.path.join(base_dir,  "matricules")
plates_folder = os.path.join(base_dir, "License Plates")
csv_path = os.path.join(plates_folder, "_annotations.csv")

# =========================================
# 2️⃣ Charger CSV
# =========================================
df = pd.read_csv(csv_path)
plates_df = df[df["class"] == "license-plate"].copy()

print("Nombre total d’annotations :", len(df))
print("Nombre de plaques trouvées :", len(plates_df))

# =========================================
# 3️⃣ Initialiser SIFT + Matcher
# =========================================
sift = cv.SIFT_create()
bf = cv.BFMatcher()

def extract_sift_from_image(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    return kp, des

# =========================================
# 4️⃣ Construire base autorisée (matricules)
# =========================================
authorized_db = []

for file in os.listdir(matricule_folder):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        path = os.path.join(matricule_folder, file)
        img = cv.imread(path)
        if img is None:
            continue

        kp, des = extract_sift_from_image(img)

        if des is not None:
            authorized_db.append({
                "filename": file,
                "image": img,
                "keypoints": kp,
                "descriptors": des
            })

print("Base autorisée construite :", len(authorized_db))

# =========================================
# 5️⃣ Vérification des voitures test
# =========================================
MATCH_THRESHOLD = 30

for _, row in plates_df.iterrows():


    for _, row in plates_df.iterrows():

        filename = row["filename"]

        xmin = int(row["xmin"])
        ymin = int(row["ymin"])
        xmax = int(row["xmax"])
        ymax = int(row["ymax"])

        # 🔴 OPTIMIZATION CONDITION
        if xmin == 0 or ymin == 0:
            print(f"⏩ Skipped {filename} (invalid bounding box)")
            continue

        path = os.path.join(plates_folder, filename)
        img = cv.imread(path)

        if img is None:
            continue

        # ✂️ Crop plate
        plate = img[ymin:ymax, xmin:xmax]

        kp_query, des_query = extract_sift_from_image(plate)

        if des_query is None:
            continue

    filename = row["filename"]
    path = os.path.join(plates_folder, filename)

    img = cv.imread(path)
    if img is None:
        continue

    # ✂️ Découper la plaque depuis le CSV
    xmin, ymin, xmax, ymax = map(int, [row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
    plate = img[ymin:ymax, xmin:xmax]

    # Extraire SIFT sur la plaque seulement
    kp_query, des_query = extract_sift_from_image(plate)

    if des_query is None:
        continue

    best_score = 0
    best_match = None
    best_good = None

    # Comparaison plaque ↔ plaques autorisées
    for item in authorized_db:

        matches = bf.knnMatch(des_query, item["descriptors"], k=2)

        good = []
        for pair in matches:
            if len(pair) == 2:
                m, n = pair
                if m.distance < 0.75 * n.distance:
                    good.append(m)

        score = len(good)

        if score > best_score:
            best_score = score
            best_match = item
            best_good = good

    # =========================================
    # Décision + Affichage
    # =========================================
    print("\n==============================")
    print("Vérification :", filename)

    if best_score >= MATCH_THRESHOLD:

        print("✅ AUTORISÉ")
        print("Correspond à :", best_match["filename"])
        print("Score :", best_score)

        # 🔹 Recalcul keypoints sur voiture complète pour affichage
        kp_full, _ = sift.detectAndCompute(img, None)

        result = cv.drawMatches(
            img, kp_full,                          # voiture complète test
            best_match["image"], best_match["keypoints"],  # matricule base
            best_good, None,
            flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        plt.figure(figsize=(12,6))
        plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
        plt.title(f"{filename} → {best_match['filename']} | Score={best_score}")
        plt.axis("off")
        plt.show()

    else:
        print("❌ REFUSÉ")
        print("Score max trouvé :", best_score)

Nombre total d’annotations : 112
Nombre de plaques trouvées : 45
Base autorisée construite : 4
⏩ Skipped cb4b754537798d23_jpg.rf.310e3c4544bf89d2f3109956b301db20.jpg (invalid bounding box)

Vérification : b6ecda23586a6ba5_jpg.rf.d737139968dd3f08447305aa7b7f6002.jpg
❌ REFUSÉ
Score max trouvé : 1
⏩ Skipped cb4b754537798d23_jpg.rf.310e3c4544bf89d2f3109956b301db20.jpg (invalid bounding box)

Vérification : b6ecda23586a6ba5_jpg.rf.d737139968dd3f08447305aa7b7f6002.jpg
❌ REFUSÉ
Score max trouvé : 1
⏩ Skipped cb4b754537798d23_jpg.rf.310e3c4544bf89d2f3109956b301db20.jpg (invalid bounding box)

Vérification : b6ecda23586a6ba5_jpg.rf.d737139968dd3f08447305aa7b7f6002.jpg
❌ REFUSÉ
Score max trouvé : 1
⏩ Skipped cb4b754537798d23_jpg.rf.310e3c4544bf89d2f3109956b301db20.jpg (invalid bounding box)

Vérification : b6ecda23586a6ba5_jpg.rf.d737139968dd3f08447305aa7b7f6002.jpg
❌ REFUSÉ
Score max trouvé : 1
⏩ Skipped cb4b754537798d23_jpg.rf.310e3c4544bf89d2f3109956b301db20.jpg (invalid bounding box)

Vérifi